In [1]:
#loading libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# setting plt to inline
%matplotlib inline

In [2]:
# importing sqlite3
import sqlite3

# conneting to db and creating cursor
conn = sqlite3.Connection('data/movies.db')
cur = conn.cursor()

# creating query functions
def fetcha(q):
    return cur.execute(q).fetchall()
def fetcho(q):
    return cur.execute(q).fetchone()

In [3]:
# getting table names
q = """SELECT name FROM sqlite_master 
WHERE type IN ('table','view') 
AND name NOT LIKE 'sqlite_%'
ORDER BY 1"""
fetcha(q)

[('bom_movie_gross',),
 ('imdb_name_basic',),
 ('imdb_name_basics',),
 ('imdb_title_akas',),
 ('imdb_title_basics',),
 ('imdb_title_crew',),
 ('imdb_title_principals',),
 ('imdb_title_ratings',),
 ('rotten_tomatoes_critic_reviews',),
 ('rotten_tomatoes_movies',),
 ('tmdb_movies',),
 ('tn_movie_budgets',)]

In [4]:
q = "PRAGMA table_info('imdb_title_principals')"
fetcha(q)

[(0, 'idx', 'INTEGER', 0, None, 1),
 (1, 'tconst', 'TEXT', 0, None, 0),
 (2, 'ordering', 'INTEGER', 0, None, 0),
 (3, 'nconst', 'TEXT', 0, None, 0),
 (4, 'category', 'TEXT', 0, None, 0),
 (5, 'job', 'TEXT', 0, None, 0),
 (6, 'characters', 'TEXT', 0, None, 0)]

In [5]:
q = "PRAGMA table_info('imdb_name_basics')"
fetcha(q)

[(0, 'idx', 'INTEGER', 0, None, 0),
 (1, 'nconst', 'TEXT', 0, None, 0),
 (2, 'primary_name', 'TEXT', 0, None, 0),
 (3, 'birth_year', 'REAL', 0, None, 0),
 (4, 'death_year', 'REAL', 0, None, 0),
 (5, 'primary_profession', 'TEXT', 0, None, 0),
 (6, 'known_for_titles', 'TEXT', 0, None, 0)]

In [6]:
q = "PRAGMA table_info('imdb_title_basics')"
fetcha(q)

[(0, 'idx', 'INTEGER', 0, None, 1),
 (1, 'tconst', 'TEXT', 0, None, 0),
 (2, 'primary_title', 'TEXT', 0, None, 0),
 (3, 'original_title', 'TEXT', 0, None, 0),
 (4, 'start_year', 'INTEGER', 0, None, 0),
 (5, 'runtime_minutes', 'REAL', 0, None, 0),
 (6, 'genres', 'TEXT', 0, None, 0)]

In [7]:
# getting column names
q = "PRAGMA table_info('bom_movie_gross')"
fetcha(q)

[(0, 'idx', 'INTEGER', 0, None, 1),
 (1, 'title', 'TEXT', 0, None, 0),
 (2, 'studio', 'TEXT', 0, None, 0),
 (3, 'domestic_gross', 'REAL', 0, None, 0),
 (4, 'foreign_gross', 'TEXT', 0, None, 0),
 (5, 'year', 'INTEGER', 0, None, 0)]

In [8]:
# creating initial dataframe by joining the four tables and printing first row
q = """SELECT*FROM imdb_name_basics
       JOIN imdb_title_principals 
       USING(nconst)
       JOIN imdb_title_basics AS ib
       USING(tconst)
       JOIN bom_movie_gross AS bm
       ON ib.primary_title = bm.title
       """
df = pd.DataFrame(fetcha(q))
df.columns = [i[0] for i in cur.description]
df.head(1)     

,idx,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles,idx,tconst,ordering,...,original_title,start_year,runtime_minutes,genres,idx,title,studio,domestic_gross,foreign_gross,year
0,0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553",466550,tt2398241,9,...,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",2804,Smurfs: The Lost Village,Sony,45000000.0,152200000,2017


In [9]:
# checking dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31120 entries, 0 to 31119
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   idx                 31120 non-null  int64  
 1   nconst              31120 non-null  object 
 2   primary_name        31120 non-null  object 
 3   birth_year          16444 non-null  float64
 4   death_year          608 non-null    float64
 5   primary_profession  30723 non-null  object 
 6   known_for_titles    30927 non-null  object 
 7   idx                 31120 non-null  int64  
 8   tconst              31120 non-null  object 
 9   ordering            31120 non-null  int64  
 10  category            31120 non-null  object 
 11  job                 10368 non-null  object 
 12  characters          12469 non-null  object 
 13  idx                 31120 non-null  int64  
 14  primary_title       31120 non-null  object 
 15  original_title      31120 non-null  object 
 16  star

In [10]:
df.foreign_gross = pd.to_numeric(df.foreign_gross, errors='coerce')
df.foreign_gross = df.foreign_gross.fillna(0)
df.domestic_gross = df.domestic_gross.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31120 entries, 0 to 31119
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   idx                 31120 non-null  int64  
 1   nconst              31120 non-null  object 
 2   primary_name        31120 non-null  object 
 3   birth_year          16444 non-null  float64
 4   death_year          608 non-null    float64
 5   primary_profession  30723 non-null  object 
 6   known_for_titles    30927 non-null  object 
 7   idx                 31120 non-null  int64  
 8   tconst              31120 non-null  object 
 9   ordering            31120 non-null  int64  
 10  category            31120 non-null  object 
 11  job                 10368 non-null  object 
 12  characters          12469 non-null  object 
 13  idx                 31120 non-null  int64  
 14  primary_title       31120 non-null  object 
 15  original_title      31120 non-null  object 
 16  star

In [11]:
# checking numbers of NaNs
df.isna().sum()

idx                       0
nconst                    0
primary_name              0
birth_year            14676
death_year            30512
primary_profession      397
known_for_titles        193
idx                       0
tconst                    0
ordering                  0
category                  0
job                   20752
characters            18651
idx                       0
primary_title             0
original_title            0
start_year                0
runtime_minutes        1021
genres                  145
idx                       0
title                     0
studio                   30
domestic_gross            0
foreign_gross             0
year                      0
dtype: int64

In [12]:
# dropping extraneous columns
df = df.drop(['idx', 'nconst', 'primary_profession', 'primary_title', 'original_title', 'known_for_titles', 'tconst', 'ordering', 'job', 'characters', 'start_year','runtime_minutes'], axis=1)
df.head(1)

,primary_name,birth_year,death_year,category,genres,title,studio,domestic_gross,foreign_gross,year
0,Mary Ellen Bauder,NaN,NaN,producer,"Adventure,Animation,Comedy",Smurfs: The Lost Village,Sony,45000000.0,152200000.0,2017


In [13]:
# creating a total_gross column
df['total_gross'] = df.domestic_gross + df.foreign_gross
df.head(1)

,primary_name,birth_year,death_year,category,genres,title,studio,domestic_gross,foreign_gross,year,total_gross
0,Mary Ellen Bauder,NaN,NaN,producer,"Adventure,Animation,Comedy",Smurfs: The Lost Village,Sony,45000000.0,152200000.0,2017,197200000.0


In [14]:
df.category.unique()

array(['producer', 'composer', 'actor', 'director', 'cinematographer',
       'writer', 'actress', 'production_designer', 'self', 'editor',
       'archive_footage', 'archive_sound'], dtype=object)

In [42]:
title_gross = df[['title', 'total_gross', 'year']]
unique_titles = title_gross.groupby('title').first().reset_index()
top50_grossing_films = unique_titles.sort_values(by='total_gross', ascending=False).head(50)
top50_grossing_films.head()

,title,total_gross,year
223,Avengers: Age of Ultron,1.405400e+09,2015
311,Black Panther,1.347000e+09,2018
1763,Star Wars: The Last Jedi,1.332600e+09,2017
1032,Jurassic World: Fallen Kingdom,1.309500e+09,2018
734,Frozen,1.276400e+09,2013


In [47]:
years1 = top50_grossing_films.year.unique()
print(years1)
print(len(years1))
np.sort(years1)

[2015 2018 2017 2013 2016 2011 2012 2014 2010]
9


array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018], dtype=int64)

In [26]:
actor_df = df[df['category'].str.contains('actor')]
actor_df.head(1)

,primary_name,birth_year,death_year,category,genres,title,studio,domestic_gross,foreign_gross,year,total_gross
10,Matt Bomer,1977.0,NaN,actor,"Comedy,Drama,Music",Magic Mike XXL,WB,66000000.0,56500000.0,2015,122500000.0


In [44]:
top_actors_films = actor_df.sort_values(by=['total_gross'], ascending=False).head(50)
top_actors_films[['primary_name', 'title', 'total_gross', 'year']].head()

,primary_name,title,total_gross,year
6661,Chris Hemsworth,Avengers: Age of Ultron,1.405400e+09,2015
11514,Robert Downey Jr.,Avengers: Age of Ultron,1.405400e+09,2015
5556,Mark Ruffalo,Avengers: Age of Ultron,1.405400e+09,2015
10408,Chris Evans,Avengers: Age of Ultron,1.405400e+09,2015
21696,Chadwick Boseman,Black Panther,1.347000e+09,2018


In [49]:
years2 = top_actors_films.year.unique()
print(years2)
print(len(years2))
np.sort(years2)

[2015 2018 2017 2013 2016 2011 2012 2014 2010]
9


array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018], dtype=int64)

In [41]:
top20_grossing_actors = actor_df.groupby(['primary_name']).total_gross.sum().sort_values(ascending=False).head(20)
top20_grossing_actors

primary_name
Robert Downey Jr.    6.843800e+09
Dwayne Johnson       5.674800e+09
Chris Hemsworth      4.841100e+09
Chris Evans          4.512106e+09
Johnny Depp          4.188000e+09
Mark Ruffalo         4.130746e+09
Mark Wahlberg        4.028500e+09
Vin Diesel           3.876200e+09
Chris Pratt          3.835803e+09
Liam Hemsworth       3.777500e+09
Ian McKellen         3.712600e+09
Steve Carell         3.604519e+09
Josh Hutcherson      3.575596e+09
Tom Cruise           3.400000e+09
Bradley Cooper       3.328580e+09
Woody Harrelson      3.306473e+09
Ryan Reynolds        3.272213e+09
Martin Freeman       3.127949e+09
Andy Serkis          3.078200e+09
Samuel L. Jackson    3.008031e+09
Name: total_gross, dtype: float64

In [50]:
actress_df = df[df['category'].str.contains('actress')]
actress_df.head()

,primary_name,birth_year,death_year,category,genres,title,studio,domestic_gross,foreign_gross,year,total_gross
26,Agata Buzek,1976.0,NaN,actress,"Drama,History",The Innocents,MBox,1100000.0,0.0,2016,1100000.0
27,Anita Caprioli,1973.0,NaN,actress,Drama,Corpo Celeste,FM,8900.0,0.0,2012,8900.0
28,Valentina Carnelutti,1973.0,NaN,actress,"Comedy,Drama",Like Crazy,ParV,3400000.0,336000.0,2011,3736000.0
130,Kate McKinnon,1984.0,NaN,actress,"Adventure,Animation,Comedy",Ferdinand,Fox,84400000.0,211700000.0,2017,296100000.0
131,Kate McKinnon,1984.0,NaN,actress,"Comedy,Crime,Thriller",Rough Night,Sony,22100000.0,25200000.0,2017,47300000.0


In [52]:
top_actresses_films = actress_df.sort_values(by=['total_gross'], ascending=False).head(50)
top_actresses_films[['primary_name', 'title', 'total_gross', 'year']].head()

,primary_name,title,total_gross,year
22982,Danai Gurira,Black Panther,1.347000e+09,2018
21255,Lupita Nyong'o,Black Panther,1.347000e+09,2018
2620,Carrie Fisher,Star Wars: The Last Jedi,1.332600e+09,2017
28562,Daisy Ridley,Star Wars: The Last Jedi,1.332600e+09,2017
8560,Bryce Dallas Howard,Jurassic World: Fallen Kingdom,1.309500e+09,2018


In [51]:
top20_grossing_actresses = actress_df.groupby(['primary_name']).total_gross.sum().sort_values(ascending=False).head(20)
top20_grossing_actresses

primary_name
Jennifer Lawrence      4.653512e+09
Anne Hathaway          3.519832e+09
Emma Stone             3.369300e+09
Kristen Wiig           3.324740e+09
Scarlett Johansson     3.232600e+09
Cate Blanchett         3.126966e+09
Zoe Saldana            2.568500e+09
Amy Adams              2.544844e+09
Angelina Jolie         2.520338e+09
Emily Blunt            2.450122e+09
Bryce Dallas Howard    2.337700e+09
Rachel McAdams         2.193292e+09
Sandra Bullock         2.167700e+09
Gwyneth Paltrow        2.043110e+09
Gal Gadot              2.038900e+09
Miranda Cosgrove       2.005600e+09
Jessica Chastain       1.991358e+09
Kristen Bell           1.762799e+09
Mila Kunis             1.760303e+09
Judi Dench             1.727900e+09
Name: total_gross, dtype: float64

In [53]:
director_df = df[df['category'].str.contains('director')]
director_df.head()

,primary_name,birth_year,death_year,category,genres,title,studio,domestic_gross,foreign_gross,year,total_gross
12,Gilles Bourdos,1963.0,NaN,director,"Biography,Drama,History",Renoir,Gold.,2300000.0,0.0,2013,2300000.0
13,David Bowers,NaN,NaN,director,"Comedy,Family",Diary of a Wimpy Kid: Rodrick Rules,Fox,52700000.0,19700000.0,2011,72400000.0
14,David Bowers,NaN,NaN,director,"Comedy,Family",Diary of a Wimpy Kid: Dog Days,Fox,49000000.0,28100000.0,2012,77100000.0
15,David Bowers,NaN,NaN,director,"Comedy,Family",Diary of a Wimpy Kid: The Long Haul,Fox,20700000.0,19400000.0,2017,40100000.0
31,Derek Cianfrance,1974.0,NaN,director,"Drama,Romance",Blue Valentine,Wein.,9700000.0,2600000.0,2010,12300000.0


In [55]:
top_directors_films = director_df.sort_values(by=['total_gross'], ascending=False).head(50)
top_directors_films[['primary_name', 'title', 'total_gross', 'year']].head()

,primary_name,title,total_gross,year
14088,Joss Whedon,Avengers: Age of Ultron,1.405400e+09,2015
25963,Ryan Coogler,Black Panther,1.347000e+09,2018
920,Rian Johnson,Star Wars: The Last Jedi,1.332600e+09,2017
19944,J.A. Bayona,Jurassic World: Fallen Kingdom,1.309500e+09,2018
21133,Jennifer Lee,Frozen,1.276400e+09,2013


In [57]:
top20_grossing_directors = director_df.groupby(['primary_name']).total_gross.sum().sort_values(ascending=False).head(20)
top20_grossing_directors

primary_name
Christopher Nolan    3.115800e+09
Peter Jackson        2.953500e+09
Michael Bay          2.833300e+09
Pierre Coffin        2.548700e+09
Joe Russo            2.546411e+09
Anthony Russo        2.546411e+09
Francis Lawrence     2.542400e+09
Zack Snyder          2.429500e+09
Bryan Singer         2.393000e+09
Chris Renaud         2.389400e+09
James Wan            2.237700e+09
Kyle Balda           2.194200e+09
Brad Bird            2.146600e+09
Rich Moore           2.024200e+09
Ridley Scott         1.992300e+09
Sam Mendes           1.989300e+09
Steven Spielberg     1.938500e+09
Lee Unkrich          1.874100e+09
Mike Mitchell        1.767400e+09
Justin Lin           1.758200e+09
Name: total_gross, dtype: float64

In [58]:
producer_df = df[df['category'].str.contains('producer')]
producer_df.head()

,primary_name,birth_year,death_year,category,genres,title,studio,domestic_gross,foreign_gross,year,total_gross
0,Mary Ellen Bauder,NaN,NaN,producer,"Adventure,Animation,Comedy",Smurfs: The Lost Village,Sony,45000000.0,152200000.0,2017,197200000.0
4,Terry Benedict,NaN,NaN,producer,"Biography,Drama,History",Hacksaw Ridge,LGF,67200000.0,108100000.0,2016,175300000.0
5,Richard Berge,NaN,NaN,producer,Documentary,The Island President,Gold.,76400.0,0.0,2012,76400.0
6,Richard Berge,NaN,NaN,producer,Documentary,An Inconvenient Sequel: Truth to Power,Par.,3500000.0,1900000.0,2017,5400000.0
7,Sam Bisbee,NaN,NaN,producer,"Comedy,Drama,Romance",Infinitely Polar Bear,SPC,1400000.0,0.0,2015,1400000.0


In [60]:
top_producers_films = producer_df.sort_values(by=['total_gross'], ascending=False).head(50)
top_producers_films[['primary_name', 'title', 'total_gross', 'year']].head()

,primary_name,title,total_gross,year
12696,Kevin Feige,Avengers: Age of Ultron,1.405400e+09,2015
12695,Kevin Feige,Black Panther,1.347000e+09,2018
10942,Ram Bergman,Star Wars: The Last Jedi,1.332600e+09,2017
5111,Kathleen Kennedy,Star Wars: The Last Jedi,1.332600e+09,2017
20589,Belén Atienza,Jurassic World: Fallen Kingdom,1.309500e+09,2018


In [61]:
top20_grossing_producers = producer_df.groupby(['primary_name']).total_gross.sum().sort_values(ascending=False).head(20)
top20_grossing_producers

primary_name
Janet Healy               4.674700e+09
Peter Chernin             4.215900e+09
Kevin Feige               3.800700e+09
Lorenzo di Bonaventura    3.595100e+09
Nina Jacobson             3.455500e+09
Charles Roven             3.276200e+09
Emma Thomas               3.115800e+09
Jon Kilik                 3.031200e+09
Christopher Meledandri    2.948400e+09
Carolynne Cunningham      2.935500e+09
Jason Blum                2.876453e+09
Ian Bryce                 2.856400e+09
Neal H. Moritz            2.746405e+09
Simon Kinberg             2.718700e+09
Don Murphy                2.711200e+09
Tom DeSanto               2.695800e+09
Patrick Crowley           2.490400e+09
J.J. Abrams               2.380500e+09
Kathleen Kennedy          2.283700e+09
Michael De Luca           2.250976e+09
Name: total_gross, dtype: float64